# Settings

In [ ]:
is_plot = True
is_export = True

landmarks_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/landmark/landmark.pkl'
meshes_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/mesh/'

start=0
stride = 1
end=120

export_folder = "../output/animate/"

# Data loading

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 3', 'marker 14', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=120/stride, contour=contour, clip_bound='xy', margin=30)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls,
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
from mesh4d import obj4d

body4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
body4d.add_obj(*body_ls)
body4d.load_markerset('landmarks', landmarks)

In [ ]:
from mesh4d import obj4d

breast4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
breast4d.add_obj(*breast_ls)
breast4d.load_markerset('landmarks', landmarks)

# Data visualisation

## 4D body

In [ ]:
if is_export:
    body4d.animate(output_folder=export_folder, filename='body4d', elements='m')

In [ ]:
if is_export:
    body4d.animate(output_folder=export_folder, filename='body4d_kps')

In [ ]:
if is_plot:
    body4d.show(elements='m', window_size=[2000, 500], zoom_rate=5, skip=12, m_props={'opacity': 0.5}, is_save=is_export, export_folder=export_folder, export_name='body4d_stack')

In [ ]:
if is_plot:
    body4d.show(elements='mk', window_size=[2000, 500], zoom_rate=5, skip=12, m_props={'opacity': 0.5}, is_save=is_export, export_folder=export_folder, export_name='body4d_kps_stack')

## 4D breast

In [ ]:
if is_export:
    breast4d.animate(output_folder=export_folder, filename='breast4d', elements='m')

In [ ]:
if is_export:
    breast4d.animate(output_folder=export_folder, filename='breast4d_kps')

In [ ]:
import os
import mesh4d

if is_plot:
    import pyvista as pv

    stack_dist = 1000
    zoom_rate = 5
    window_size = [2000, 500]
    
    scene = pv.Plotter()
    plot_num = len(body4d.obj_ls)
    skip = 12

    for idx in range(0, plot_num, skip):
        body = body4d.obj_ls[idx]
        breast = breast4d.obj_ls[idx]
        width = body.get_width()

        body.add_mesh_to_scene(scene, location=[0, 0, idx * stack_dist / skip], opacity=0.1)
        breast.add_mesh_to_scene(scene, location=[0, 0, idx * stack_dist / skip])
        breast.add_kps_to_scene(scene, location=[0, 0, idx * stack_dist / skip], radius=0.02*width)
        
    scene.camera_position = 'zy'
    scene.camera.azimuth = 45
    scene.camera.zoom(zoom_rate)
    scene.window_size = window_size
    scene.enable_parallel_projection()
    scene.show(interactive_update=True)

    if is_export:
        export_path = os.path.join(export_folder, 'breast_crop_stack.png')
        scene.update()
        scene.screenshot(export_path)
        
        if mesh4d.output_msg:
            print("export image: {}".format(export_path))

